In [2]:
import torch
import numpy as np

In [3]:
mat = np.load("viper//viper_features.npy", allow_pickle=True)

In [4]:
EPS = 0.001 # the regularization constant
sigma = 2**-16 # the kernel width
N = 632 # number of persons
d = 100 # number of features

### Loading parameters

In [5]:
idxa = mat.item().get('idxa')-1
idxa = idxa.astype('float64')
idxa = torch.from_numpy(idxa)

idxb = mat.item().get('idxb')-1
idxb = idxb.astype('float64')
idxb = torch.from_numpy(idxb)

X = mat.item().get('ux')[:d, :] 
X = torch.from_numpy(X)

print(idxa.shape, idxb.shape, X.shape)

torch.Size([1, 632]) torch.Size([1, 632]) torch.Size([100, 1264])


### Data Review

In [6]:
X[:,0]

tensor([ 5.6763e+01, -2.1639e+01, -2.6869e+01, -2.6857e+01, -1.2353e+01,
         9.6739e+00, -2.4680e+00, -4.1892e+01, -6.1503e+00, -1.8926e+01,
        -8.1701e-01,  3.4005e+01,  1.1992e+01, -3.7692e+01,  1.2938e+01,
         1.9612e+01, -4.2859e-01,  2.0604e+01, -4.4400e+00,  8.6628e+00,
        -1.4934e+01, -1.7920e+01, -2.0539e+00,  5.2963e+00, -1.0537e+01,
        -1.1793e+01, -1.3612e+01, -4.4364e+00, -3.4284e+00, -3.7028e+00,
        -9.3622e+00,  6.6155e+00,  1.9674e+01, -9.0171e-01, -8.4710e+00,
         9.8897e-01, -7.4855e+00,  1.9710e+01, -5.8935e+00,  6.1913e+00,
         2.1174e+00, -8.7990e+00,  5.6792e+00, -1.3161e+01,  2.8724e-01,
        -5.4849e+00, -4.0117e+00,  6.0003e+00, -8.6844e+00,  2.5930e+00,
        -5.0884e+00,  4.9267e+00, -2.0084e+01,  1.0568e+01, -1.3966e+01,
         5.3753e+00, -6.1388e+00,  5.9289e+00, -4.0125e+00,  1.8924e+00,
        -3.0191e+00,  5.9632e+00, -2.4214e+00,  9.8576e-01, -2.9458e+00,
         3.2906e+00, -3.8012e+00,  1.4486e+01, -3.5

In [7]:
perm = torch.randperm(N)

In [8]:
idxtrain = perm[:N//2]
idxtest = perm[N//2:]

In [9]:
idxtrain.shape

torch.Size([316])

In [10]:
first_ind = torch.cat((idxa[0,idxtrain], idxa[0, idxtrain]))
first_ind = first_ind.view(1, first_ind.shape[0])
second_ind = torch.cat((idxb[0, idxtrain], idxb[0, idxtrain[torch.randperm(N//2)]] ))
second_ind = second_ind.view(1, second_ind.shape[0])

In [11]:
print(first_ind.shape, second_ind.shape)

torch.Size([1, 632]) torch.Size([1, 632])


In [12]:
temp_a = torch.ones(N)
temp_b = torch.cat((torch.ones(N//2), torch.zeros(N//2)))
matches = torch.logical_and(temp_a, temp_b)

In [13]:
S = torch.cat((first_ind[:, matches], second_ind[:, matches]))
D = torch.cat((first_ind[:, torch.logical_not(matches)], second_ind[:, torch.logical_not(matches)]))

### Compute kernelmatrix

In [14]:
from kernelmatrix import kernelmatrix
K = kernelmatrix(X=X, X2=torch.tensor([]), sigma=sigma)

### Compute H

In [15]:
n = K.shape[0]
B = torch.zeros(n, 1)
E = torch.zeros(n, 1)
W = torch.zeros(n, n)

In [16]:
ib = D[0, :].type(torch.int64)
# .view(1, D.shape[1])
ie = D[1, :].type(torch.int64)
# .view(1, D.shape[1])
print(ib.shape, ie.shape)

torch.Size([316]) torch.Size([316])


In [17]:
t = torch.ones(316,1)
print(t.shape)
# print(t)

torch.Size([316, 1])


In [18]:
B[ib.type(torch.int64)] = t

In [19]:
E[ie.type(torch.int64)] = t

In [20]:
for i in range(0, ib.shape[0]):
    W[ib[i], ie[i]] = W[ib[i],ie[i]]+1

In [21]:
from computeH import computeH
H = computeH(1264, ib=D[0,:], ie=D[1,:])

In [22]:
torch.count_nonzero(torch.diagonal(H))
# matlab output nnz(diag(H0)): 632
# NO DOUBT we're calculating H's correctly.

tensor(632)

### kernel function


In [23]:
n = K.shape[0]
e = EPS
n1 = S.shape[1]
n0 = D.shape[1]
H0 = computeH(n, D[0, :], D[1, :])
H1 = computeH(n, S[0, :], S[1, :])

In [24]:
from helps.kernel import computeT
C = computeT(n0, 0.001, H0, K)-computeT(n1, 0.001, H1, K)

In [25]:
torch.count_nonzero(C)

tensor(399424)

In [26]:
torch.diagonal(C)[:100]

tensor([   0.0000,  -16.5693, -390.1460,    0.0000, -223.1174,    0.0000,
         -84.4830,    0.0000,    0.0000,    0.0000,   18.8604,    0.0000,
        -228.7330,  222.7599,    0.0000,    0.0000, -133.6817,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
        -238.3554, -118.5372, -161.4838,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,  312.6203, -184.7387,  -17.4127,
        -182.7256,    0.0000,    0.0000,  447.9342,  -55.4967, -244.7327,
        -400.4982,    0.0000, -177.8339,    0.0000,    0.0000,    0.0000,
        -107.3422,    0.0000,    0.0000,    0.0000, -153.5592,    0.0000,
           0.0000, -122.8219, -201.0423, -193.3706,    0.0000,    0.0000,
          -4.1882, -214.9554, -280.9341, -312.6125, -305.2442,  273.6842,
         -96.2715,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
        -179.1505,    0.0000,    0.0000,  213.1648,  -47.8891,    0.0000,
         -10.7244,    0.0000,  -45.450

In [27]:
# from kernelmatrix import kernelmatrix

# K  = kernelmatrix(X=X, X2=torch.tensor([]), sigma=sigma);       

# part_A = (1/(n*EPS**2))
# print(part_A)

# t = torch.eye(K.shape[0]) + (1/(n*2))*torch.matmul(K.double(), H.double())
# torch.count_nonzero(t)

# part_B = torch.inverse(t)
# print(torch.count_nonzero(part_B))

# C = part_A * torch.matmul(H.double(), part_B)
# torch.count_nonzero(C)

In [28]:
from helps.kernel import kernel
M = kernel(EPS, S, D, K, pmetric=1)
print(M.shape)
# print(torch.diagonal(M)[:100])
# print(M[:100,:10])

torch.Size([1264, 1264])


### project to positive-semidefinite cone

In [29]:
# a = torch.tensor([[0.6529, 0.1343, 0.1778], [0.4367, 0.5539, 0.9764], [0.5284, 0.6233, 0.3749]])
# print(a.shape)
# print(a)
# x, y = torch.eig(a)
# print(x)
# print(y)

In [30]:
D, V = torch.eig(M, eigenvectors=True)
d = D[:,0]
for idx, i in enumerate(d):
    if i<=0:
        d[idx] = torch.finfo(torch.float64).eps

In [31]:
D_ = torch.diag(d).type(torch.double)
V = V.type(torch.double)
M = torch.mm(V, torch.mm(D_, V.T))
# torch.diagonal(M)[1:100]

### Pairwise Distance

In [32]:
import torch

In [33]:
data = X
idxa = idxa.type(torch.int64).T
idxb = idxb.type(torch.int64).T
print(idxa.shape, idxb.shape)

torch.Size([632, 1]) torch.Size([632, 1])


In [34]:
# Retrieve p and q from the data "X"
p = data[:, idxa[idxtest]]
p = p.view(p.shape[0], -1)
q = data[:, idxb[idxtest]]
q = q.view(q.shape[0], -1)
print(p.shape, q.shape)

torch.Size([100, 316]) torch.Size([100, 316])


In [35]:
(d, pn) = p.shape
(d, qn) = q.shape

### E.g.

In [36]:
a = torch.randn(4, 4)
print(a)

tensor([[ 0.8016,  1.2166,  0.3805, -0.4936],
        [-0.3568, -0.7677, -0.5663, -1.1806],
        [ 0.6396,  0.0568, -0.5527, -1.8268],
        [-0.6032, -1.0041,  0.5980,  0.6830]])


In [37]:
torch.sum(torch.square(torch.floor(a)), 0)

tensor([2., 6., 2., 9.])

### Calculate $ \|x_{i}\|^2 $ & $ \|x_{j}\|^2 $


In [38]:
pmag = torch.sum(torch.square(p), 0, keepdim=True).double()
qmag = torch.sum(torch.square(q), 0, keepdim=True).double()
print(pmag.shape, qmag.shape)

torch.Size([1, 316]) torch.Size([1, 316])


### Calculate $ \|x_{i}-x_{j}\|^2 $ 

### Simplification: $ \|x_{i}-x_{j}\|^2 \rightarrow \|x_{i}-x{j}\|^T\|x_{i}-x_{j}\| \rightarrow \|x_{i}\|^2 + \|x_{j}\|^2 - 2x_{i}^Tx_{j}  $ 


In [39]:
m = torch.mm(torch.ones(pn, 1).double(), qmag) + torch.mm(pmag.T, torch.ones(1, qn).double()) - 2 * torch.mm(p.T, q)

In [40]:
from helps.sqdist import sqdist
m_ = sqdist(p, q)
print(m[0, 0], m_[0, 0])

tensor(39311.7711, dtype=torch.float64) tensor(39311.7711, dtype=torch.float64)


### Mahalanobis Distance Metric

In [41]:
A = M
data = K
p = data[:, idxa[idxtest]]
p = p.view(p.shape[0], -1)
q = data[:, idxb[idxtest]]
q = q.view(q.shape[0], -1)
print(p.shape, q.shape, A.shape)

torch.Size([1264, 316]) torch.Size([1264, 316]) torch.Size([1264, 1264])


In [42]:
Ap = torch.mm(A, p)
Aq = torch.mm(A, q)
pmag = torch.sum(p*Ap, 0, keepdim=True)
qmag = torch.sum(q*Aq, 0, keepdim=True)

In [43]:
print(pmag[:,1:100])

tensor([[220.6418, 256.4801, 298.7390, 276.0280, 256.8567, 235.7673, 295.9084,
         296.4245, 256.3360, 295.2847, 240.9109, 211.1132, 207.1084, 205.6746,
         285.1915, 218.0659, 255.8814, 331.0374, 191.9767, 233.1496, 291.8562,
         298.6010, 221.7205, 234.2675, 213.8361, 250.4603, 237.5133, 234.4653,
         239.6698, 323.1029, 292.2321, 287.3912, 252.8718, 248.8953, 267.1580,
         166.9565, 285.7725, 197.5639, 293.9923, 280.0688, 244.6829, 212.8363,
         323.4734, 299.4401, 303.9126, 246.0818, 230.3990, 202.3866, 193.5801,
         282.8243, 206.6331, 335.9902, 263.3602, 249.5412, 353.9243, 262.1608,
         267.5490, 249.6633, 247.7452, 206.2337, 279.8859, 277.6253, 260.0620,
         209.5269, 235.3947, 238.9673, 316.2528, 203.5449, 204.6538, 227.5491,
         287.5737, 271.6425, 228.9560, 285.9964, 343.8030, 324.4986, 313.0709,
         257.1539, 344.5227, 184.9295, 249.9111, 276.9733, 276.7288, 277.1230,
         297.5389, 238.1275, 311.6183, 196.5360, 297

In [44]:
m = torch.mm(torch.ones(pn, 1).double(), qmag) + torch.mm(pmag.T, torch.ones(1, qn).double()) - 2 * torch.mm(p.T, q)

In [45]:
print(m.shape)
print(torch.diagonal(m)[1:100])

torch.Size([316, 316])
tensor([-226.7618,  140.6194,   45.5721,    0.7358,  214.5834,  268.8083,
         157.8075, -147.4658,  281.5724,  111.0422, -165.0020,   56.8357,
         132.6237,   74.8466, -166.6706,  261.7201,  -55.4505,   12.9445,
        -128.8495,  224.7504, -177.1359,  110.9231,  109.3424,   55.4743,
         -62.8355,  -69.3029,  199.0281,  215.0192, -199.5359,    9.4712,
         102.6151, -218.4428,  260.9415,  170.4952,  145.8677,  127.0024,
         122.4612, -127.1304,  104.2311,  192.7935,   32.0578,  189.9127,
          -7.9455,   84.6412,   37.1567, -166.4969,  243.4067, -109.1262,
         166.8554,  244.9288,   53.6378,   42.9516,  -84.1291,   75.3682,
         112.9847, -147.6092,  154.9232, -117.0829,  196.9501,  177.9805,
         -54.0046,  -22.4030,  100.6101,  232.8323, -114.4569,   34.6232,
          97.2929,  -45.0178, -182.0398,  158.6138,  146.0001, -164.1137,
        -164.6437,   60.6331,  145.6544,   34.6893,  217.5118, -215.1500,
         269.05

In [46]:
from helps.sqdist import mahalanobis_dist
m_ = mahalanobis_dist(p, q, A)
print(m_.shape)

torch.Size([316, 316])


In [47]:
m_[0:100, 0]

tensor([ -6.0036, -37.8354,  54.9305, 135.2983,  72.6150, 229.7414, 248.5374,
         47.5404, -44.7362, 257.9386, 121.0423, -92.3173,  76.6056, 140.9949,
        -43.6102, -34.3224,  55.2140,  44.4430,  77.6330, -55.3194, 199.3310,
        -67.1114, 150.6204, -79.6539, 149.4422, -45.0100, -82.7248, 123.0108,
         59.4252, -39.8556,  23.0221, 109.3217, -65.1611, 151.4707, 157.5959,
         76.1185, 194.4682,  44.8793, -33.3637, 103.7782, 166.4990,  78.6275,
         80.0475,  58.9328, 143.4761,  25.4634, -87.0066, 106.1612, -87.3284,
        177.6944, 151.7909, -37.8034,  38.4808, -28.1840,  76.0443, 107.8021,
        -16.4743, 143.9042,  -9.1452, 175.1320,   0.9532,  35.2973, -15.3230,
        115.2094,  69.9127, -26.6850,  69.1144,  99.3742,  48.3538, -73.8189,
        -21.4805, 113.0475, -54.8157, -64.8433, 138.9112, 122.3325, 107.2457,
         97.0321, -83.9143, 203.1338, 202.6733, -62.1387, -18.6175,  84.2586,
        -10.5825, 120.1963, -79.9603,   4.8887, -23.5329, -15.66

In [48]:
from helps.calcMCMC import calcMCMC
dist = calcMCMC(M, K, idxa, idxb, idxtest)

In [49]:
dist[0,0], m_[0,0]
print(dist.shape)

torch.Size([1, 316])


In [50]:
idxtest.shape

torch.Size([316])

### Calculate Rank 1 and k-Kissme

In [51]:
result = torch.zeros(1, dist.shape[1])
for pairCounter in range(0, dist.shape[1]):
    distPair = dist[pairCounter, :]
    tmp, idx = torch.sort(distPair)
    result[:, idx==pairCounter] = result[:, idx==pairCounter] + 1
        
print(result)

IndexError: index 1 is out of bounds for dimension 0 with size 1